In [1]:
import pandas as pd
import re
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from textblob import TextBlob
import wordsegment as ws 
ws.load()
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

In [2]:
data_entertainment = pd.read_json('tweets_entertainment.json', lines = True)
data_entertainment.sort_values('text',inplace=True)
data_entertainment.drop_duplicates(subset='text',keep = False, inplace=True)
data_entertainment.shape

(4675, 36)

In [3]:
def clean_tweets(df):
  i = 1  
  all_tweets = []
  tweets = df['text'].values.tolist();
  for text in tweets:
    print()
    print(i)
    i +=1
    print(text)
    text = text.lower()
    text_pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]|))+')
    text = text_pattern.sub('', text)
    emoji = re.compile("["
                        u"\U0001F600-\U0001FFFF"   #EMOJIS
                        u"\U0001F300-\U0001F5FF"   #Symbols and Pictographs
                        u"\U0001F680-\U0001F6FF"   #map symbols
                        u"\U0001F1E0-\U0001F1FF"   #flags
                        u"\U00002702-\U000027B0"
                        u"\U000024C2-\U0001F251"
                        "]+", flags=re.UNICODE)
    text = emoji.sub(r'', text)
    text = re.sub(r'@[a-z0-9]+','',text)
   # text = re.sub(r'#', '', text)
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"this's", "this is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"\'ll", "will", text)
    text = re.sub(r"\'ve", "have", text)
    text = re.sub(r"\'m", "am", text)
    text = re.sub(r"\'d", "would", text)
    text = re.sub(r"\'re", "are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"don't", "do not", text)
    text = re.sub(r"didn't", "did not", text)
    text = re.sub(r"can't", "can not", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"couldn't", "could not", text)
    text = re.sub(r"haven't", "have not", text)
    text = re.sub(r'rt[\s]+', '', text)

    text = re.sub(r"[,.\"!#$%^&*(){}?/;'~:<>+=-]", "", text)
    
    tokens = word_tokenize(text)
    
    text = [ws.clean(w) for w in text if len(ws.clean(w)) > 0]
    text = [ws.segment(w)[0] for w in text]   #segmentation
    
    stop_words = set(stopwords.words("english"))
    stop_words.discard("not")
    table = str.maketrans('','',string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]

    words = [w for w in words if not w in stop_words]
    words = ' '.join(words)
        
    print(words)
    
    all_tweets.append(words)
  return all_tweets

In [5]:
nan_value = float("NaN")

Cleaned_tweets = clean_tweets(data_entertainment)

data_entertainment['text_cleaned'] = Cleaned_tweets

data_entertainment.replace("", nan_value, inplace=True)
data_entertainment.dropna(subset=['text_cleaned'], inplace=True)

data_entertainment = data_entertainment.sample(frac = 1)

data_entertainment = data_entertainment[data_entertainment['text_cleaned'].str.split().str.len().gt(4)]

data_entertainment = data_entertainment[['id', 'text_cleaned','text']]

data_entertainment.to_csv('tweets_cleaned_entertainment.csv',index=False)


1
!!!YESPLEASE
yesplease

2
"#Entertainment News: Big Hit #Entertainment And Universal Music Group Announce Expanded Strategic Partnership - PR… https://t.co/qT0HQcltDw
entertainment news big hit entertainment universal music group announce expanded strategic partnership

3
"#Entertainment News: Southeast Michigan entertainment calendar Feb. 19 and beyond - The Macomb Daily #News": https://t.co/Tt9UDPQM9U
entertainment news southeast michigan entertainment calendar feb beyond macomb daily news

4
"#Entertainment News: Sycamore #Entertainment and SEGI TV Are Officially Approved As an Amdocs/Vubiquity Affiliate… https://t.co/YSBQBVwvru
entertainment news sycamore entertainment segi tv officially approved amdocsvubiquity

5
"#Entertainment News: Today's events for Feb. 18 | #Entertainment | https://t.co/pZMXmf4ywt - Charleston Post Couri… https://t.co/stwaZVbkj0
entertainment news todays events feb entertainment charleston post

6
"...He's never fought us. Not us united."

Celebrating th

johnkagang versus emraankiforce adrenaline pumping dialogues plot ampamp performances give

94
#JusticeLeague #SnyderCut is here! @HBOAsia @HBO https://t.co/NHEFiFLzJe
justiceleague snydercut

95
#JusticeLeague amazing story telling.  Great work by Synder and why creatives should be able to finish the story th… https://t.co/Bl4tPfRROj
justiceleague amazing story telling great work synder creatives able finish story

96
#JusticeLeague star Jared Leto is now selling "We Live in a Society" T-shirts with 50% of the profits benefitting t… https://t.co/QlFEBLZ1dx
justiceleague star jared leto selling live society tshirts profits benefitting

97
#JusticeLeagueSnyderCut is 🔥.
Now this is the JL movie we deserved.
Thank you, @ZackSnyder! #JusticeLeague #SnyderCut
justiceleaguesnydercut jl movie deserved thank justiceleague snydercut

98
#JusticeLeagueSnyderCut ✅ #FalconAndWinterSoldier 🔜 We being fed.
justiceleaguesnydercut falconandwintersoldier fed

99
#KajalAggarwal title cars as Kajal Aggar

rajkummarrao janhvikapoor varunsharma talk celeb reactions roohi exclusive

205
#ReleaseReminder 
#MumbaiSaga Starring @TheJohnAbraham @emraanhashmi @SunielVShetty @prateikbabbar @rohitroy500… https://t.co/rsP94k6QRN
releasereminder mumbaisaga starring

206
#Roohi #BoxOffice: The horror comedy collects Rs. 1.02 cr. on Day 8

https://t.co/Ji9FgGSdzr
roohi boxoffice horror comedy collects rs cr day

207
#Roohi BOC

Day 8 - ₹ 1.02cr

Total - ₹ 17.43cr
roohi boc day total

208
#Roohi Day 8: 1.00 Cr Net

Total Net: 16.50 Cr
roohi day cr net total net cr

209
#Roohi First Week (8 Days) Box Office Collection Report, Score Well on 2nd Thursday, Overall Decent! 
https://t.co/kQ4XhjDsiJ
roohi first week days box office collection report score well thursday overall decent

210
#Roohi Week1 17.5cr+ is very good considering #COVID19 &amp; 50% occupancy limit. Could’ve done 30cr+ in normal days🤘… https://t.co/3DaxWbJlNG
roohi good considering amp occupancy limit could done normal

211
#Roohi goes st

snyder cut spoilers wait years everyone like man darkseid cgi

309
*SNYDER CUT SPOILERS*

If that’s supposed to be Ares I guess this means that this before he decided to go strictly… https://t.co/iUyvGzNQva
snyder cut spoilers supposed ares guess means decided go

310
*SNYDER CUT SPOILERS*

Is it still considered a TRIdent if it has five points?

#SnyderCut   #JusticeLeague #DCEU #Spoilers
snyder cut spoilers still considered trident five points snydercut justiceleague dceu spoilers

311
*SNYDER CUT SPOILERS*

Remember how much everyone complained about how Cyborg’s CG looked in the theatrical cut?… https://t.co/ItXHyhS3vj
snyder cut spoilers remember much everyone complained cyborg cg looked theatrical

312
*SNYDER CUT SPOILERS*

Save for the cheesy song (again) that was a pretty cool Flash scene. I hope we get to see mo… https://t.co/MMA4s463IV
snyder cut spoilers save cheesy song pretty cool flash scene hope get see

313
*SNYDER CUT SPOILERS*

You know Barry, you might be able to ma

twt thank masterpiece namjoon remedy

424
@Binged_ Bro when #FalconAndWinterSoldier it will stream exact time
bro falconandwintersoldier stream exact time

425
@Binged_ where is #FalconAndWinterSoldier  Its streming Tonight Or not 😕🙁

Plz clarify Its to sleepy 😴😴
falconandwintersoldier streming tonight not plz clarify sleepy

426
@Bizzlelover4 @wehavethetea @PopCrave @justinbieber Yet again you proved my point- nothing about this album was abo… https://t.co/ITxYKoOj9j
yet proved point nothing album

427
@BlackMajikMan90 Can we agree that when Bucky eventually admits to the old man what he did to his son it’ll be a cr… https://t.co/5CBkaDZRbO
agree bucky eventually admits old man son

428
@BoyzinhaC justin bieber
justin bieber

429
@BuckyIsotope Bucky, its my birthday too. Can you print me a copy? Thanks buddy
bucky birthday print copy thanks buddy

430
@CHA0SCHAE RM WE ARE SO PROUD OF YOU 
#RMixtape_6주년_축하해 #넌_RM냥_너를해줘 #6YearsOfRMixtape #6YearsWithDoYou #4YearsWithChange #RM @BTS_twt
r

...

bylac love celebrity

4142
What makes it worse is that half way through there is ANOTHER sample of an MLK speech! In between an album full of… https://t.co/M7rhfx7CW6
makes worse half way another sample mlk speech album full

4143
What should a diverse Justice League of America (that actually reflects America's demographics) look like?… https://t.co/VlVNO55lcg
diverse justice league america actually reflects americas demographics look

4144
What type of announcement we are going to get from @iamsrk @yrf 
Is that a video about happening there. 
Let's wait… https://t.co/5ZRG06oNuW
type announcement going get video happening lets

4145
What's bylac???

IU5 IS COMING!
#IU #아이유 #BYLAC https://t.co/BghuE2O2td
bylac coming iu bylac

4146
What's convenient about the #SnyderCut is I remember literally nothing from the original #JusticeLeague. https://t.co/oDyCifOd4B
convenient snydercut remember literally nothing original justiceleague

4147
What? What?!
I'm late for the party..

How do you spe

addison rae justin bieber lana del rey obsessed justice chemtrails country club song album rating white

4258
after celebrity meron agad bago @jrmjsprvelasco 👀
celebrity meron agad bago

4259
all I wanted was for bucky to be okay and steve did everything in his power to stop that
wanted bucky okay steve everything power stop

4260
all i want do is hug bucky and kiss his big ass head
want hug bucky kiss big ass head

4261
all that promo and sam and bucky weren’t even together
promo sam bucky even together

4262
also Bucky is drinking Tiger https://t.co/HcAY2TNL6E
also bucky drinking tiger

4263
am trying not to spoil anything for #FalconAndWinterSoldier but, i can't help saying, 
it pained me that Marvel tre… https://t.co/QaL3cjumPT
trying not spoil anything falconandwintersoldier not help saying pained marvel

4264
and if she really has her hair dyed then what??

then she goes back to black again for filming, and we won't even g… https://t.co/1qegW2TUAu
really hair dyed goes back black

go subscribe youtube youtube contentcreator subscribe

4384
https://t.co/ydXbTzqSO4

2021-FEB-17 3:25pm
Consumer Services · up 0.03% = $3.0 billion gain.

#consumerServices… https://t.co/2xdo0CyCaB
consumer services billion gain

4385
huhuhu kaya pla busy si IU may pa-comeback na naman ulit😭💜😍

CAN'T REALLY WAIT! I THINK THIS CONCEPT IS FOR HER BIR… https://t.co/R0jADBRKRe
huhuhu kaya pla busy si iu may pacomeback na naman ulit not really wait think concept

4386
i am james bucky barnes and you’re part of my efforts to make amends. 🙂
james bucky barnes paof efforts make amends

4387
i can’t tell how excited i am for this, i’ve been waiting for so long
tell excited waiting long

4388
i didn't know i could love bucky any more than i already did but then he went and bought those flowers
not know could love bucky already went bought flowers

4389
i didnt even see this 😀 feel attacked from all angles rn
didnt even see feel attacked angles rn

4390
i don’t care how much i like someone i woul

omg made day

4513
omo. it will be a blast. great year to start with you 
IU 5th Album Teaser 'BYLAC'

#아이유 #IU #BYLAC
omo blast great year stawith iu album teaser bylac iu bylac

4514
one of the most surreal voices in our industry. so proud of her.
SHREYA GHOSAL ON BILLBOARD.
one surreal voices industry proud shreya ghosal billboard

4515
only bad bitches have sam or bucky disney+ icons 🥵 #FalconAndTheWinterSoldier https://t.co/beqGvOUZg5
bad bitches sam bucky disney icons falconandthewintersoldier

4516
oomfs said bucky is ugly???
oomfs said bucky ugly

4517
pandemic-quality content
pandemicquality content

4518
playing on https://t.co/aWiBnEN765 is #FooFighters ... check out my review of their album #MedicineAtMidnight:… https://t.co/Rb9Jw7Epe9
playing foofighters check review album

4519
please .... No
please

4520
please don’t sound like celebrity
please sound like celebrity

4521
please imagine them calling each other and talking about random things to keep each other company 🥺
p

shout family

4636
“no” “okay , okay” SIR. PLEASE MARRY ME 😍
okay okay sir please marry

4637
…
We're waiting.
We're waiting eagerly.
#Pathan will strike like a bomb. 💣💖
#ShahRukhKhan
weare waiting weare waiting eagerly pathan strike like bomb shahrukhkhan

4638
‼️TFATWS SPOILERS‼️
-
-
All I can think about is Bucky potentially losing Yori and having to deal with that grief a… https://t.co/3c4zNTGieE
think bucky potentially losing yori deal grief

4639
⍟ the falcon and the winter soldier spoilers ✪
.
.
.
.
bucky has been controlled for so long that he doesn't even k… https://t.co/W3Vp7wK12w
falcon winter soldier spoilers bucky controlled long doesnt even

4640
⚡️⚡️#ZoeyDeutch, #OliviaCooke, #JenniferLawrence, #YaraShahidi, #KellyMarieTran, #BeckyG, and #IsabelaMerced were c… https://t.co/OBR5tLiyC8
zoeydeutch oliviacooke jenniferlawrence yarashahidi kellymarietran beckyg isabelamerced

4641
✨✨✨ Justice de Justin Bieber✨✨✨
justice de justin bieber

4642
❤ We love her #YouTube #music #en

In [6]:
data_entertainment

,id,text_cleaned,text
4737,1372620648128864260,watching snydercut justiceleague hbomax hashta...,RT @TheKloudXCVI: Watching #SnyderCut #Justice...
14175,1372985696777027591,rm proud rm rm twt,@iK00MlMI RM WE ARE SO PROUD OF YOU \n#RMixtap...
8793,1372963810433064964,hi please play anyone justin bieber thank much...,"Hi @OnAirRomeo @MostRequestLive, please can yo..."
2510,1362191578593304581,years ago thefugees released thescore album ge...,25 years ago #TheFugees released #TheScore. Th...
806,1362124060302462986,many oneoff events forced make tough decisions...,Many one-off events forced to make tough decis...
...,...,...,...
11769,1372972757881819137,already time flies fast rm rmixtape,6years already ? time flies so fast . \n\n#RMi...
3703,1370042322906279938,thank much seokjin masterpiece jin seokjin bts...,@STRBRRY8 thank you so much seokjin for this m...
6654,1372630609772810243,gift stpatricksday saleuse reusable silicone l...,RT @topreuse2: #gift #StPatricksDay #Sale.Use ...
2405,1362187640146178051,lockdown blues story lockdown around world tol...,Marathon85- The Lockdown Blues!! The story of ...
